<a href="https://colab.research.google.com/github/fedorst/brain-visualization-utcg2020/blob/main/kym_vision_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
path = "/content/drive/My Drive/dataEngProject/kym_vision.json"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_json(path, orient="index")

In [ ]:
df["url"] = df.index
df = df.reset_index()
df = df.drop(columns=["index"])
df.head()

,labelAnnotations,safeSearchAnnotation,webDetection,error,url
0,"[{'mid': '/m/01yrx', 'description': 'Cat', 'sc...","{'adult': 'VERY_UNLIKELY', 'spoof': 'VERY_LIKE...","{'webEntities': [{'entityId': '/m/04wpw', 'sco...",NaN,https://knowyourmeme.com/memes/this-is-relevan...
1,"[{'mid': '/m/0dzct', 'description': 'Face', 's...","{'adult': 'UNLIKELY', 'spoof': 'LIKELY', 'medi...","{'webEntities': [{'entityId': '/m/03bzkh4', 's...",NaN,https://knowyourmeme.com/memes/leave-britney-a...
2,"[{'mid': '/m/068jd', 'description': 'Photograp...","{'adult': 'UNLIKELY', 'spoof': 'VERY_LIKELY', ...","{'webEntities': [{'entityId': '/m/09dzg', 'sco...",NaN,https://knowyourmeme.com/memes/i-like-turtles
3,"[{'mid': '/m/09j2d', 'description': 'Clothing'...","{'adult': 'VERY_UNLIKELY', 'spoof': 'VERY_LIKE...","{'webEntities': [{'entityId': '/m/09kqc', 'sco...",NaN,https://knowyourmeme.com/memes/bitches-dont-know
4,"[{'mid': '/m/04hgtk', 'description': 'Head', '...","{'adult': 'VERY_UNLIKELY', 'spoof': 'VERY_LIKE...","{'webEntities': [{'entityId': '/m/06ng23', 'sc...",NaN,https://knowyourmeme.com/memes/o-rly


In [ ]:
safeSearchToNumberCode = {
    "VERY_UNLIKELY": 0.0,
    "UNLIKELY": 0.25,
    "POSSIBLE": 0.5,
    "LIKELY": 0.75,
    "VERY_LIKELY": 1.0
}
def getSafeSearch(row):
  if (isinstance(row.safeSearchAnnotation, dict)):
    for key in row.safeSearchAnnotation.keys():
      row["safeSearch_"+key] = safeSearchToNumberCode[row.safeSearchAnnotation[key]]
  return row

In [ ]:
df = df.apply(getSafeSearch, axis=1)
df = df.drop(columns=["safeSearchAnnotation"])

In [ ]:
df.head()

,error,labelAnnotations,safeSearch_adult,safeSearch_medical,safeSearch_racy,safeSearch_spoof,safeSearch_violence,url,webDetection
0,NaN,"[{'mid': '/m/01yrx', 'description': 'Cat', 'sc...",0.00,0.00,0.00,1.00,0.25,https://knowyourmeme.com/memes/this-is-relevan...,"{'webEntities': [{'entityId': '/m/04wpw', 'sco..."
1,NaN,"[{'mid': '/m/0dzct', 'description': 'Face', 's...",0.25,0.25,0.25,0.75,0.25,https://knowyourmeme.com/memes/leave-britney-a...,"{'webEntities': [{'entityId': '/m/03bzkh4', 's..."
2,NaN,"[{'mid': '/m/068jd', 'description': 'Photograp...",0.25,0.25,0.00,1.00,0.25,https://knowyourmeme.com/memes/i-like-turtles,"{'webEntities': [{'entityId': '/m/09dzg', 'sco..."
3,NaN,"[{'mid': '/m/09j2d', 'description': 'Clothing'...",0.00,0.00,0.25,1.00,0.25,https://knowyourmeme.com/memes/bitches-dont-know,"{'webEntities': [{'entityId': '/m/09kqc', 'sco..."
4,NaN,"[{'mid': '/m/04hgtk', 'description': 'Head', '...",0.00,0.25,0.00,1.00,0.50,https://knowyourmeme.com/memes/o-rly,"{'webEntities': [{'entityId': '/m/06ng23', 'sc..."


In [ ]:
df.safeSearch_adult.value_counts()

0.00    12710
0.25     3331
0.50      376
0.75      115
1.00       39
Name: safeSearch_adult, dtype: int64

In [ ]:
def getLabelAnnotScores(row):
  if ((not type(row.labelAnnotations) == float) and len(row.labelAnnotations) > 0):
    #newDict = {annot["description"]: annot["score"] for annot in row.labelAnnotations}
    #row["annot_array"] = list(newDict.keys())
    row["annot_array"] = [annot["description"] for annot in row.labelAnnotations]
  return row

In [ ]:
def getBestGuessLabel(row):
  if ((type(row.webDetection) == dict)):
    # and "bestGuessLabels" in row.webDetection and len(row.webDetection.bestGuessLabels) > 0):
    bestGuessLabels = row.webDetection.get("bestGuessLabels", [])
    if (len(bestGuessLabels) > 0 and 'label' in bestGuessLabels[0]):
      row["webDetection_bestGuessLabel"] = bestGuessLabels[0]['label']
  return row

In [ ]:
df = df.apply(getLabelAnnotScores, axis=1)
df = df.drop(columns="labelAnnotations")

In [ ]:
df = df.apply(getBestGuessLabel, axis=1)
df = df.drop(columns="webDetection")

In [ ]:
df.head()

,annot_array,error,safeSearch_adult,safeSearch_medical,safeSearch_racy,safeSearch_spoof,safeSearch_violence,url,webDetection_bestGuessLabel
0,"[Cat, Felidae, Carnivore, Small to medium-size...",NaN,0.00,0.00,0.00,1.00,0.25,https://knowyourmeme.com/memes/this-is-relevan...,interest meme
1,"[Face, Forehead, Nose, Hair, Cheek, Skin, Lip,...",NaN,0.25,0.25,0.25,0.75,0.25,https://knowyourmeme.com/memes/leave-britney-a...,leave britney alone
2,"[Photograph, Light, Smile, Screenshot, News, F...",NaN,0.25,0.25,0.00,1.00,0.25,https://knowyourmeme.com/memes/i-like-turtles,like turtles
3,"[Clothing, Hairstyle, Shoulder, Muscle, Neck, ...",NaN,0.00,0.00,0.25,1.00,0.25,https://knowyourmeme.com/memes/bitches-dont-know,eric harris funny
4,"[Head, Bird, Snowy owl, Eye, Beak, Feather, Ow...",NaN,0.00,0.25,0.00,1.00,0.50,https://knowyourmeme.com/memes/o-rly,o rly


In [ ]:
df.to_parquet("/content/drive/My Drive/dataEngProject/kym_vision_processed.parquet")

In [ ]:
len(df.columns)

9

In [ ]:
df.shape

(16686, 9)